In [35]:
#!pip install ortools

import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import pickle

In [36]:
time_matrix = np.load('고연전_MVP_테스트_경로매트릭스(분).npy')

In [37]:
name_lst = ['안암역','이문동교회','여의도역','홍대입구역','동천학교','63빌딩.가톨릭대학교여의도성모병원','태릉입구역7번출구.서울생활사박물관','성원아파트경남아너스빌앞']

In [38]:
len(name_lst)

8

In [39]:
len(time_matrix)

8

In [40]:
"""Simple Pickup Delivery Problem (PDP) + Capacited Vehicles Routing Problem (CVRP)."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['time_matrix'] = time_matrix
    data['demands'] = [0]+[1 for i in range(7)]
    data['vehicle_capacities'] = [4,4]
    data['num_vehicles'] = 2
    data['depot'] = 0
    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    total_time = 0
    total_load = 0

    grouped_bus_station_lst = []
    

    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n\n'.format(vehicle_id)
        route_time = 0
        route_time_lst = [0]
        route_load = 0
        i=0
        loop_lst = []
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_time_lst.append(routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id))
            route_time += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)
            loop_lst.append(node_index)
            plan_output += ' 정류장 이름 : {0}  (탑승객 수: {1}, 하차수: {2}, 소요시간: {3}) \n    ==>    '.format(name_lst[node_index], data['vehicle_capacities'][vehicle_id] - route_load,data['demands'][node_index],datetime.timedelta(seconds = route_time_lst[i]))
            i+=1
        grouped_bus_station_lst.append(loop_lst)
        plan_output += '서비스 종료\n\n'
        plan_output += 'Duration of the route: {}분\n'.format((datetime.timedelta(seconds = route_time-route_time_lst[-1])))
        plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_time += route_time
        total_load += route_load
    print('\n')
    print('Total Duration of all routes: {}m'.format(datetime.timedelta(seconds =total_time)))
    print('Total load of all routes: {}'.format(total_load))
   
    grouped_bus_station = pd.DataFrame({'Vehicle':[i for i in range(2)],'bus_station':grouped_bus_station_lst})
    
    with open("grouped_bus_station.pickle", "wb") as f:
	    pickle.dump(grouped_bus_station, f)

def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['time_matrix']),
                                           data['num_vehicles'], data['depot'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # 1-1. Define cost of each arc.
    def distance_callback(from_index, to_index):
        """Returns the manhattan distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['time_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # 1-2.Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        8000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100) #이거 의미?

    # 2-1.Define cost of each node.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)

    # 2-2.Add Capacity constraint.
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)
    

    # Print solution on console.
    if solution:
      solution_dict =  print_solution(data, manager, routing, solution)
    else:
      print(solution)

    
  
if __name__ == '__main__':
    main()

Objective: 506104
Route for vehicle 0:

 정류장 이름 : 안암역  (탑승객 수: 4, 하차수: 0, 소요시간: 0:00:00) 
    ==>     정류장 이름 : 여의도역  (탑승객 수: 3, 하차수: 1, 소요시간: 0:20:52) 
    ==>     정류장 이름 : 홍대입구역  (탑승객 수: 2, 하차수: 1, 소요시간: 0:01:26) 
    ==>     정류장 이름 : 63빌딩.가톨릭대학교여의도성모병원  (탑승객 수: 1, 하차수: 1, 소요시간: 0:14:21) 
    ==>     정류장 이름 : 태릉입구역7번출구.서울생활사박물관  (탑승객 수: 0, 하차수: 1, 소요시간: 0:03:34) 
    ==>    서비스 종료

Duration of the route: 0:40:13분
Load of the route: 4

Route for vehicle 1:

 정류장 이름 : 안암역  (탑승객 수: 4, 하차수: 0, 소요시간: 0:00:00) 
    ==>     정류장 이름 : 이문동교회  (탑승객 수: 3, 하차수: 1, 소요시간: 0:14:40) 
    ==>     정류장 이름 : 동천학교  (탑승객 수: 2, 하차수: 1, 소요시간: 0:10:09) 
    ==>     정류장 이름 : 성원아파트경남아너스빌앞  (탑승객 수: 1, 하차수: 1, 소요시간: 0:29:26) 
    ==>    서비스 종료

Duration of the route: 0:54:15분
Load of the route: 3



Total Duration of all routes: 2:26:44m
Total load of all routes: 7


In [41]:
f = open("grouped_bus_station.pickle", "rb")
grouping  = pickle.load(f)
f.close()

grouping

,Vehicle,bus_station
0,0,"[0, 2, 3, 5, 6]"
1,1,"[0, 1, 4, 7]"


In [42]:
time_matrix

array([[   0,  880, 1252, 1416, 1060, 1572, 1619, 1823],
       [ 711,    0, 1120, 1026,  609, 1283, 1275, 1986],
       [1367, 1025,    0,   86,  594,  841,  885, 2187],
       [1358, 1016,  187,    0,  614,  861,  905, 2207],
       [ 997,  635,  511,  597,    0,  545,  589, 1766],
       [1420, 1061,  981, 1017,  653,    0,  214, 2170],
       [1418, 1054,  966, 1001,  651,  221,    0, 2163],
       [1718, 2039, 2039, 2171, 1847, 2359, 2405,    0]])

In [43]:
time_matrix = pd.DataFrame(time_matrix)
time_matrix_0 = time_matrix[grouping['bus_station'][1]].loc[grouping['bus_station'][1]]
time_matrix_0 = time_matrix_0.to_numpy().tolist()
time_matrix_0

[[0, 880, 1060, 1823],
 [711, 0, 609, 1986],
 [997, 635, 0, 1766],
 [1718, 2039, 1847, 0]]

In [44]:
name_lst = ['안암_우신향병원','이문동교회','여의도역','홍대입구역','동천학교','63빌딩.가톨릭대학교여의도성모병원','태릉입구역7번출구.서울생활사박물관','성원아파트경남아너스빌앞']
name_lst2 = [name_lst[i] for i in grouping['bus_station'][1]]
name_lst2

['안암_우신향병원', '이문동교회', '동천학교', '성원아파트경남아너스빌앞']

In [46]:
"""Simple Pickup Delivery Problem (PDP) + Capacited Vehicles Routing Problem (CVRP)."""

from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import datetime


def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data['time_matrix'] = time_matrix_0
    data['demands'] = [0,1,1,1] 
    data['vehicle_capacities'] = [4]
    data['num_vehicles'] = 1

    data['starts'] = [0]
    data['ends'] = [int(np.argmax(time_matrix_0[0]))]

    return data


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f'Objective: {solution.ObjectiveValue()}')
    total_time = 0
    total_load = 0

    for vehicle_id in range(data['num_vehicles']):
        index = routing.Start(vehicle_id)
        plan_output = 'Route for vehicle {}:\n\n'.format(vehicle_id)
        route_time = 0
        route_time_lst = [0]
        route_load = 0
        i=0

        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data['demands'][node_index]
            
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_time_lst.append(routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id))
            route_time += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id)

            plan_output += ' 정류장 이름 : {0}  (탑승객 수: {1}, 하차수: {2}, 소요시간: {3}) \n    ==>    '.format(name_lst2[node_index], route_load.sum(),data['demands'][node_index],datetime.timedelta(seconds = route_time_lst[i]))
            i+=1
        node_index = manager.IndexToNode(index)
        plan_output += ' 정류장 이름 : {0}  (탑승객 수: {1}, 하차수: {2}, 소요시간: {3}) \n    ==>    '.format(name_lst2[node_index], data['vehicle_capacities'][vehicle_id] - route_load,data['demands'][node_index],datetime.timedelta(seconds = route_time_lst[i]))
        plan_output += '서비스 종료\n\n'
        #plan_output += 'Duration of the route: {}분\n'.format((datetime.timedelta(seconds = route_time-route_time_lst[-1])))
        #plan_output += 'Load of the route: {}\n'.format(route_load)
        print(plan_output)
        total_time += route_time
        total_load += route_load
    print('\n')
    print('Total Duration of all routes: {}m'.format(datetime.timedelta(seconds =total_time)))
    print('Total load of all routes: {}'.format(total_load))
   

def main():
    """Entry point of the program."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(len(data['time_matrix']),
                                           data['num_vehicles'], data['starts'], data['ends'])

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)


    # 1-1. Define cost of each arc.
    def distance_callback(from_index, to_index):
        """Returns the manhattan distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['time_matrix'][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # 1-2.Add Distance constraint.
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        8000,  # vehicle maximum travel distance
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    distance_dimension.SetGlobalSpanCostCoefficient(100) #이거 의미?

    # 2-1.Define cost of each node.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data['demands'][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(
        demand_callback)

    # 2-2.Add Capacity constraint.
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data['vehicle_capacities'],  # vehicle maximum capacities
        True,  # start cumul to zero
        'Capacity')

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PARALLEL_CHEAPEST_INSERTION)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)
    

    # Print solution on console.
    if solution:
      solution_dict =  print_solution(data, manager, routing, solution)
    else:
      print(solution)

    
  
if __name__ == '__main__':
    main()

Objective: 328755
Route for vehicle 0:

 정류장 이름 : 안암_우신향병원  (탑승객 수: 4, 하차수: 0, 소요시간: 0:00:00) 
    ==>     정류장 이름 : 이문동교회  (탑승객 수: 3, 하차수: 1, 소요시간: 0:14:40) 
    ==>     정류장 이름 : 동천학교  (탑승객 수: 2, 하차수: 1, 소요시간: 0:10:09) 
    ==>     정류장 이름 : 성원아파트경남아너스빌앞  (탑승객 수: 2, 하차수: 1, 소요시간: 0:29:26) 
    ==>    서비스 종료




Total Duration of all routes: 0:54:15m
Total load of all routes: 2
